## Duplicate checks

In [1]:
import logging
import sys
import os

import pandas as pd
import numpy as np
from pandas import Timestamp
# pylint: disable=import-error disable=wrong-import-position
from pathlib import Path
sys.path.append(os.path.dirname(Path.cwd().parent))
import pydit
logger=pydit.setup_logging_info()


In [2]:
df=pd.read_excel("./input/data.xlsx")
df.columns

Index(['customer_id', 'gender', 'first_name', 'last_name', 'dob',
       'date_enrollment', 'city', 'country', 'residential address',
       'date_renewal', 'lifetime_id', 'registration_id'],
      dtype='object')

In [3]:
pydit.check_duplicates(df,["customer_id"])

2022-03-31 16:51 INFO - Duplicates in fields: customer_id
2022-03-31 16:51 INFO - (keep=False)
2022-03-31 16:51 INFO - 80 of population 10000


,customer_id,gender,first_name,last_name,dob,date_enrollment,city,country,residential address,date_renewal,lifetime_id,registration_id
20,21,M,Norman,Morales,2007-09-26,2019-10-19,Lake Wendy,Togo,43340 Nash Ridge Suite 093,2021-12-19,266-49197945,78-4916288210733
21,21,M,Norman,Morales,2007-09-26,2019-10-19,Lake Wendy,Togo,43340 Nash Ridge Suite 093,2021-12-19,266-49197945,78-4916288210733
34,34,F,Kimberly,Taylor,1998-08-14,2011-02-22,Hannaborough,Iceland,903 Phillips Drive,2022-04-25,054-85642973,03-8999397667052
35,34,F,Kimberly,Taylor,1998-08-14,2011-02-22,Hannaborough,Iceland,903 Phillips Drive,2022-04-25,054-85642973,03-8999397667052
145,144,F,Julie,Johnson,1985-11-07,2018-09-10,Sanchezton,Antarctica (the territory South of 60 deg S),5732 Wagner Knoll Apt. 609,2022-06-04,062-82294748,74-8215857652117
...,...,...,...,...,...,...,...,...,...,...,...,...
9787,9788,M,Ryan,Nguyen,2001-06-02,2004-12-18,Lake Donna,Trinidad and Tobago,52354 Cooper Spurs,2021-09-03,170-63505412,85-1166317253597
9882,9884,F,Audrey,Solis,2003-09-06,2001-10-19,North Jerome,Croatia,27423 Amy Parkways Apt. 438,2021-12-22,232-95915999,34-7906412092141
9883,9884,F,Lisa,Hahn,1979-07-19,2005-06-14,Bennettberg,Nauru,40138 Michael Radial,2021-12-24,232-95915999,25-5216844241867
9885,9886,M,Willie,Goodman,1999-09-15,2012-11-17,Calvinstad,Latvia,07488 Wilson Lane,2022-05-29,279-58871010,56-0130166283471
